# GAN Dataset
The dataset used to train the StyleGAN3 model was manually populated by sourcing various videos with the desired style and extracting images from the frames. <br>
Note: The videos were sourced from [Pexels](https://www.pexels.com/), which provides copyright-free content.

The following script was used to extract N frames from an input video.

In [ ]:
import os
import cv2
import random
import sys
import uuid

def crop_center_square(frame):
    """
    Crops the central square from the frame to avoid aspect ratio distortion
    when resizing (e.g., from 16:9 to 1:1).

    Args:
        frame (numpy.ndarray): The input image/frame.

    Returns:
        numpy.ndarray: The cropped square frame.
    """
    h, w = frame.shape[:2]
    min_dim = min(h, w)
    
    # Calculate center coordinates
    start_x = (w - min_dim) // 2
    start_y = (h - min_dim) // 2
    
    # Perform the crop
    return frame[start_y:start_y+min_dim, start_x:start_x+min_dim]

def extract_random_frames(video_path, output_folder, num_frames, target_size):
    """
    Extracts a specified number of random frames from a video, crops them to squares,
    resizes them, and saves them to disk.

    Args:
        video_path (str): Path to the source video file.
        output_folder (str): Directory where images will be saved.
        num_frames (int): Number of random frames to extract.
        target_size (int): Dimension (width/height) of the output square images.
    """
    if not os.path.exists(video_path):
        print(f"Error: The video file '{video_path}' does not exist.")
        return

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created directory: {output_folder}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video loaded. Total available frames: {total_frames}")

    if total_frames < num_frames:
        print(f"Warning: Video has fewer frames ({total_frames}) than requested ({num_frames}). Extracting maximum available.")
        num_frames = total_frames

    random_indices = sorted(random.sample(range(total_frames), num_frames))

    print(f"Starting extraction of {num_frames} frames...")
    
    saved_count = 0
    
    for idx in random_indices:
        # Seek directly to the specific frame index
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        
        ret, frame = cap.read()
        
        if ret:
            # Center Square Crop
            cropped_frame = crop_center_square(frame)
            
            # Resize
            resized_frame = cv2.resize(cropped_frame, (target_size, target_size), interpolation=cv2.INTER_AREA)
            
            # Save
            filename = os.path.join(output_folder, f"{str(uuid.uuid4())}.jpg")
            cv2.imwrite(filename, resized_frame)
            
            saved_count += 1
            
            if saved_count % 10 == 0:
                sys.stdout.write(f"\rExtracted: {saved_count}/{num_frames}")
                sys.stdout.flush()
        else:
            print(f"\nUnable to read frame at index {idx}")

    cap.release()
    print(f"\n\nFinished! {saved_count} images saved in '{output_folder}'.")

# GAN Training

The dataset consists of 1,095 images. The StyleGAN3 repository was downloaded, and training was initiated. The GAN training ran for approximately 24 hours until a model capable of generating high-quality images was obtained.
<br><br>
The training was started running the train.py script inside StyleGAN3 repository. Check [here](https://github.com/NVlabs/stylegan3?tab=readme-ov-file#training) for more info.